In [1]:
from milvus import default_server
from pymilvus import connections, utility

## Connects to a server:

In [2]:
default_server.start()



    __  _________ _   ____  ______
   /  |/  /  _/ /| | / / / / / __/
  / /|_/ // // /_| |/ / /_/ /\ \
 /_/  /_/___/____/___/\____/___/ {Lite}

 Welcome to use Milvus!

 Version:   v2.2.8-lite
 Process:   49802
 Started:   2023-06-08 10:48:14
 Config:    /home/nam/.milvus.io/milvus-server/2.2.8/configs/milvus.yaml
 Logs:      /home/nam/.milvus.io/milvus-server/2.2.8/logs

 Ctrl+C to exit ...


In [3]:
connections.connect(host='0.0.0.0', port=19530)

print(utility.get_server_version())

default_server.stop()

v2.2.9


In [4]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

In [5]:
connections.connect(host='0.0.0.0', port=19530)

## Creates a collection:

In [6]:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="random", dtype=DataType.DOUBLE),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=8)
]
schema = CollectionSchema(fields, "hello_milvus is the simplest demo to introduce the APIs")
hello_milvus = Collection("hello_milvus", schema)

In [7]:
hello_milvus

<Collection>:
-------------
<name>: hello_milvus
<description>: hello_milvus is the simplest demo to introduce the APIs
<schema>: {'auto_id': False, 'description': 'hello_milvus is the simplest demo to introduce the APIs', 'fields': [{'name': 'pk', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'random', 'description': '', 'type': <DataType.DOUBLE: 11>}, {'name': 'embeddings', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 8}}]}

## Inserts vectors in the collection:

In [ ]:
import random
entities = [
    [i for i in range(3000)],  # field pk
    [float(random.randrange(-20, -10)) for _ in range(3000)],  # field random
    [[random.random() for _ in range(8)] for _ in range(3000)],  # field embeddings
]
insert_result = hello_milvus.insert(entities)
hello_milvus.flush()  

In [14]:
print(connections.list_connections())

[('default', <pymilvus.client.grpc_handler.GrpcHandler object at 0x7f20f5651850>)]


## Builds indexes on the entities:

In [15]:
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
hello_milvus.create_index("embeddings", index)

Status(code=0, message=)

## Loads the collection to memory and performs a vector similarity search:

In [29]:
hello_milvus.load()
vectors_to_search = entities[-1][-2:]
vectors_to_search

[[0.9967219718601805,
  0.1407229715135655,
  0.9034955954499895,
  0.23132348750172516,
  0.30521265694946564,
  0.8210684870032136,
  0.9351960407007905,
  0.20340311178037374],
 [0.48375748380765604,
  0.04786951894646152,
  0.1347254989840232,
  0.7687886656416449,
  0.2518474480016062,
  0.14094864110650707,
  0.35747443103454046,
  0.5500142310862204]]

In [61]:

search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 12},
}
results = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=5, output_fields=["pk","random"])

In [62]:
for i, result in enumerate(results):
    print("\nSearch result for {}th vector: ".format(i))
    for j, res in enumerate(result):
        print("Top {}: {}".format(j, res))


Search result for 0th vector: 
Top 0: id: 2998, distance: 0.0, entity: {'pk': 2998, 'random': -17.0}
Top 1: id: 2582, distance: 0.13441693782806396, entity: {'pk': 2582, 'random': -19.0}
Top 2: id: 1617, distance: 0.1445169448852539, entity: {'pk': 1617, 'random': -13.0}
Top 3: id: 198, distance: 0.16958031058311462, entity: {'pk': 198, 'random': -17.0}
Top 4: id: 142, distance: 0.17570573091506958, entity: {'pk': 142, 'random': -11.0}

Search result for 1th vector: 
Top 0: id: 2999, distance: 0.0, entity: {'pk': 2999, 'random': -18.0}
Top 1: id: 2646, distance: 0.07510833442211151, entity: {'pk': 2646, 'random': -15.0}
Top 2: id: 1278, distance: 0.16128209233283997, entity: {'pk': 1278, 'random': -11.0}
Top 3: id: 823, distance: 0.16161221265792847, entity: {'pk': 823, 'random': -16.0}
Top 4: id: 2431, distance: 0.17484146356582642, entity: {'pk': 2431, 'random': -13.0}


## Performs a vector query:

In [63]:
result = hello_milvus.query(expr="random > -12", output_fields=["random", "embeddings"])
result

[{'random': -11.0,
  'embeddings': [0.59757024,
   0.18332824,
   0.64325076,
   0.62063533,
   0.11719743,
   0.1733666,
   0.75012356,
   0.26791516],
  'pk': 22},
 {'random': -11.0,
  'embeddings': [0.7293103,
   0.4290586,
   0.026304606,
   0.7440281,
   0.55470705,
   0.73574644,
   0.9350738,
   0.30223262],
  'pk': 37},
 {'random': -11.0,
  'embeddings': [0.5269975,
   0.15906179,
   0.96843094,
   0.18195567,
   0.86484563,
   0.112985596,
   0.54913867,
   0.9814851],
  'pk': 70},
 {'random': -11.0,
  'embeddings': [0.80789715,
   0.7538679,
   0.16838291,
   0.71195143,
   0.043595828,
   0.17616238,
   0.5291589,
   0.7549691],
  'pk': 85},
 {'random': -11.0,
  'embeddings': [0.1687176,
   0.36768952,
   0.7497381,
   0.50264573,
   0.18656239,
   0.2044102,
   0.9948669,
   0.49218106],
  'pk': 88},
 {'random': -11.0,
  'embeddings': [0.94804186,
   0.23018596,
   0.72967327,
   0.36559704,
   0.2087047,
   0.7139898,
   0.10594244,
   0.93108267],
  'pk': 90},
 {'random':

## Performs a hybrid search:

In [64]:
results = hello_milvus.search(vectors_to_search, "embeddings", search_params, 
                              limit=3, expr="random > -12", output_fields=["random"])

for i, result in enumerate(results):
    print("\nSearch result for {}th vector: ".format(i))
    for j, res in enumerate(result):
        print("Top {}: {}".format(j, res))


Search result for 0th vector: 
Top 0: id: 142, distance: 0.17570573091506958, entity: {'random': -11.0}
Top 1: id: 2094, distance: 0.3171996474266052, entity: {'random': -11.0}
Top 2: id: 581, distance: 0.3270356357097626, entity: {'random': -11.0}

Search result for 1th vector: 
Top 0: id: 1278, distance: 0.16128209233283997, entity: {'random': -11.0}
Top 1: id: 718, distance: 0.27020204067230225, entity: {'random': -11.0}
Top 2: id: 1649, distance: 0.2990214228630066, entity: {'random': -11.0}


## Drops the collection:


In [ ]:
utility.drop_collection("hello_milvus")